# 멀티에이전트 크롤링 파이프라인 구축
앞선 실습에서 우리는 두 가지 전문 에이전트를 설계했습니다.
- **Navigator** (02): 브라우저를 조작하여 웹사이트 구조를 분석하고 CSS 셀렉터를 찾아내는 탐색 전문가
- **Coder** (03): Blueprint를 받아 실제로 동작하는 크롤링 코드를 작성하고 스스로 디버깅하는 개발 전문가

이번 실습에서는 이 두 에이전트를 **멀티에이전트 워크플로우** 위에서 함께 작업하도록 구현합니다.

Navigator와 Coder는 각자의 역할에만 집중하고, 두 에이전트 사이에는 **Blueprint(청사진)** 라는 구조화된 JSON 객체가 오갑니다. Blueprint에는 수집할 URL, 계층별 CSS 셀렉터, 페이지네이션 방식, JS 렌더링 여부 등이 담겨 있어, Coder는 HTML을 직접 보지 않고도 정확한 크롤링 코드를 작성할 수 있습니다.




👤 사용자: "이 사이트에서 데이터 수집해줘" ↓ 🗺️ Navigator ──(Blueprint)──▶ 💻 Coder ──▶ 📄 수집 결과


### 이번 실습에서 배우는 것

1. **동적 N계층 Blueprint 스키마** 설계 — 1~N개의 탐색 계층을 유연하게 표현하는 Pydantic 모델
2. **Navigator 에이전트 고도화** — `get_page_structure` (HTML 구조 분석) + `browse_web` (동적 인터랙션) 이중 도구 체계
3. **Coder 에이전트 구현** — Blueprint 해석 → Playwright 코드 작성 → 실행 → 오류 시 셀프 디버깅
4. **파이프라인 연결** — Navigator → Coder 전체 흐름 자동화
5. **실습 과제** — LangGraph `StateGraph`로 파이프라인을 더욱 견고하게 구조화

In [ ]:
import os, sys
from dotenv import load_dotenv

load_dotenv(override=True)

# PROJECT_ROOT를 .env에서 읽기 (없으면 현재 디렉토리)
project_root = os.getenv("PROJECT_ROOT", os.getcwd())

# 프로젝트 루트가 유효하지 않으면, 현재 위치에서 상위로 찾기
if not os.path.exists(os.path.join(project_root, "app")):
    # 상위 디렉토리 탐색
    current = os.getcwd()
    for _ in range(5):
        if os.path.exists(os.path.join(current, "app")):
            project_root = current
            break
        current = os.path.dirname(current)

# Working Directory 설정
os.chdir(project_root)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import app

print(f"✅ Project Root: {project_root}")
print(f"✅ Working Directory: {os.getcwd()}")

In [ ]:
import asyncio
from browser_use import Agent, Browser, ChatGoogle

import nest_asyncio

# 주피터 노트북 환경에서 비동기 루프 중복 실행 오류를 방지하기 위해 사용합니다.
nest_asyncio.apply()

# 가상 디스플레이(noVNC) 번호를 지정합니다.
# 이 설정이 있어야 GUI 브라우저가 가상 화면(:1)에 나타납니다.
#os.environ["DISPLAY"] = ":1"

# 동적 N계층 Blueprint 스키마 설계

핵심 아이디어: **"계층을 레고 블록처럼, 필요한 만큼만 쌓는다"**

---

## 클래스 구조

NavigatorBlueprintCollection ← Navigator의 최종 출력 (1개 이상의 Blueprint 묶음) 

└── blueprints: list[NavigatorBlueprint] 

└── layers: list[PageLayer] ← 탐색 계층 하나를 표현하는 단위 블록



##### `PageLayer` - 탐색 계층 단위 블록

| 필드 | 설명 |
|------|------|
| `layer_name` | 이 계층의 역할 이름 (예: '기사 목록', '상품 상세') |
| `url_pattern` | 이 계층의 URL 구조 예시 |
| `selectors` | `{필드명: CSS셀렉터}` 딕셔너리 |
| `navigate_to_next` | 다음 계층 링크의 CSS 셀렉터. **마지막 계층이면 반드시 `None`** |
| `pagination_method` | 페이지네이션 방식 (URL파라미터 / AJAX버튼 / 무한스크롤 / `None`) |

**`navigate_to_next` 필드가 핵심입니다:**
- 값이 있으면 → 그 CSS 셀렉터로 링크를 추출하여 다음 계층으로 이동
- `None`이면 → 이 계층이 마지막 (데이터 수집 완료)

##### `NavigatorBlueprint` - 단일 수집 작업 설계도

| 필드 | 설명 |
|------|------|
| `entry_urls` | 크롤링 시작 URL 목록. 구조가 같고 시작점만 다른 경우 여러 개 지정 |
| `total_layers` | 탐색에 필요한 총 계층 수 (layers 리스트 길이와 동일) |
| `layers` | 탐색 순서대로 정렬된 `PageLayer` 목록 |
| `rendering_type` | `"Static SSR"` 또는 `"Dynamic CSR/JS"` |
| `anti_bot_notes` | 로그인, 팝업, 캡차 등 주의사항. 없으면 `"없음"` |

##### `NavigatorBlueprintCollection` - Navigator의 최종 출력

| 필드 | 설명 |
|------|------|
| `total_jobs` | 총 Blueprint 수 |
| `blueprints` | Blueprint 목록 |

**Blueprint 개수 결정 기준:**
- 구조(계층/셀렉터)가 같고 시작 URL만 다른 경우 → **Blueprint 1개**, `entry_urls`에 복수 URL
- 구조가 근본적으로 다른 경우 (다른 사이트 등) → **Blueprint 별도 생성**

---

## 스키마 검증 예시 3가지

| 예시 | 설명 |
|------|------|
| 예시 1 | Blueprint 1개, `entry_urls` 1개 (단일 섹션) |
| 예시 2 | Blueprint 1개, `entry_urls` 2개 (구조 동일, URL만 다름) |
| 예시 3 | Blueprint 2개 (사이트 구조가 달라 분리) |


In [ ]:
# navigator 모듈 임포트
from browser_use import Browser
from notebooks.navigator import (
    navigator_agent,
    save_blueprints,
    NavigatorContext,
    PageLayer,
    NavigatorBlueprint,
    NavigatorBlueprintCollection)

In [ ]:
import json
from typing import Optional
from pydantic import BaseModel, Field, field_validator

# ==========================================
# ✅ 스키마 검증 예시
# ==========================================

# 예시 1: 단일 섹션 (네이버 금융 주식 시세) - Blueprint 1개, entry_urls 1개
sample_single = NavigatorBlueprintCollection(
    total_jobs=1,
    blueprints=[
        NavigatorBlueprint(
            entry_urls=["https://finance.naver.com/sise/sise_rise.nhn"],
            total_layers=1,
            layers=[
                PageLayer(
                    layer_name="주식 상승 종목 테이블",
                    url_pattern="https://finance.naver.com/sise/sise_rise.nhn",
                    selectors={"stock_name": "td.col_type1 a", "change_rate": "td.rate_t"},
                    navigate_to_next=None,
                    pagination_method="URL파라미터"
                )
            ],
            rendering_type="Static SSR",
            anti_bot_notes="없음"
        )
    ]
)

# 예시 2: 동일 구조, 출발 URL만 다른 경우 (정치면 + 사회면) - Blueprint 1개, entry_urls 2개
sample_multi_url = NavigatorBlueprintCollection(
    total_jobs=1,
    blueprints=[
        NavigatorBlueprint(
            entry_urls=[
                "https://news.naver.com/section/100",  # 정치
                "https://news.naver.com/section/102",  # 사회
            ],
            total_layers=2,
            layers=[
                PageLayer(
                    layer_name="뉴스 목록",
                    url_pattern="https://news.naver.com/section/{sid}",
                    selectors={"article_link": "a.sa_text_title"},
                    navigate_to_next="a.sa_text_title",
                    pagination_method="AJAX버튼"
                ),
                PageLayer(
                    layer_name="기사 상세",
                    url_pattern="https://n.news.naver.com/mnews/article/{mid}/{aid}",
                    selectors={"title": "h2.media_end_head_headline", "body": "#dic_area", "datetime": ".media_end_head_info_dateline"},
                    navigate_to_next=None
                )
            ],
            rendering_type="Dynamic CSR/JS",
            anti_bot_notes="더보기 버튼 AJAX 방식"
        )
    ]
)

# 예시 3: 구조가 다른 두 사이트 - Blueprint 2개
sample_multi_blueprint = NavigatorBlueprintCollection(
    total_jobs=2,
    blueprints=[
        NavigatorBlueprint(
            entry_urls=["https://news.naver.com/section/100"],
            total_layers=2,
            layers=[
                PageLayer(layer_name="뉴스 목록", url_pattern="https://news.naver.com/section/100",
                          selectors={"article_link": "a.sa_text_title"}, navigate_to_next="a.sa_text_title"),
                PageLayer(layer_name="기사 상세", url_pattern="https://n.news.naver.com/mnews/article/{mid}/{aid}",
                          selectors={"title": "h2.media_end_head_headline", "body": "#dic_area"}, navigate_to_next=None)
            ],
            rendering_type="Dynamic CSR/JS", anti_bot_notes="없음"
        ),
        NavigatorBlueprint(
            entry_urls=["https://www.danawa.com/product/?cateCode=112758"],
            total_layers=2,
            layers=[
                PageLayer(layer_name="상품 목록", url_pattern="https://www.danawa.com/product/?cateCode=112758",
                          selectors={"product_link": "a.prod_name"}, navigate_to_next="a.prod_name", pagination_method="URL파라미터"),
                PageLayer(layer_name="상품 상세", url_pattern="https://prod.danawa.com/info/?pcode={id}",
                          selectors={"name": "h3.prod_tit", "price": "span.low_price", "specs": "div.spec_list"}, navigate_to_next=None)
            ],
            rendering_type="Dynamic CSR/JS", anti_bot_notes="없음"
        )
    ]
)

In [ ]:
print("✅ 예시 1 (단일):")
print(sample_single.model_dump_json(indent=2))


In [ ]:
print("\n✅ 예시 2 (동일 구조, URL 2개):")
print(sample_multi_url.model_dump_json(indent=2))

In [ ]:
print("\n✅ 예시 3 (다른 구조 사이트 2개):")
print(sample_multi_blueprint.model_dump_json(indent=2))

# Navigator 에이전트 재설계

왜 `get_page_structure` + `browse_web`를 함께 사용하는가?

웹 데이터를 수집할 때 가장 어려운 문제는  
**"어디에서 데이터를 가져와야 하는지(구조 이해)"** 와  
**"실제로 데이터를 가져오는 것(행동 실행)"** 이 서로 다른 문제라는 점입니다.

우리는 이 두 문제를 하나의 도구로 해결하지 않고,  
**서로 역할이 다른 두 개의 도구로 분리**하여 해결합니다.

---

## 1️⃣ get_page_structure — 구조 분석 도구 (Planning Tool)

`get_page_structure`는 웹페이지의 **HTML 구조 일부를 토큰 효율적으로 반환**합니다.

이 도구의 목적은 **데이터 수집 계획(청사진)을 만드는 것**입니다.

### 역할
- 페이지의 HTML 구조 확인
- class / id 패턴 분석
- 반복되는 데이터 영역 발견 (뉴스 목록, 상품 리스트 등)
- 안정적인 CSS selector 설계

즉,
> "어디에서 데이터를 가져와야 하는가?"를 이해하는 단계입니다.

⚠️ 중요한 점:
- browser agent는 화면 의미를 이해하지만
- **재사용 가능한 CSS selector 설계에는 HTML 근거가 필요**합니다.

`get_page_structure`는 바로 이 근거를 제공합니다.

---

## 2️⃣ browse_web — 실행 도구 (Execution Tool)

`browse_web`는 browser-use 기반 에이전트를 이용해 실제 브라우저 행동을 수행합니다.

### 역할
- 클릭
- 스크롤
- 페이지 이동
- 동적 콘텐츠 로딩
- 실제 데이터 확인

즉,
> "계획한 위치에서 실제로 데이터를 가져오는 단계"입니다.

browser-use는 사람처럼 페이지를 이해하고 행동하는 데 강하지만,
기본적으로 **셀렉터 추출기(selector extractor)가 아니라 실행기(executor)** 입니다.

---

### 네비게이터 테스트

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

async def main():
    # ──────────────────────────────────────────────────────
    # 테스트 (1): 정치 섹션 기사 제목 5개 수집
    # ──────────────────────────────────────────────────────
    print("=" * 60)
    print("🧪 테스트 (1): 정치 섹션 기사 제목 5개 수집")
    print("=" * 60)

    # Runtime Context용 공유 브라우저 인스턴스 생성
    shared_browser_instance = Browser(
        headless=False,
        disable_security=True,
        keep_alive=True,
    )

    # Thread ID 설정 및 브라우저를 Context에 주입
    nav_config_1 = {"configurable": {"thread_id": "nav_test_2_refined"}}
    nav_context_1 = NavigatorContext(shared_browser=shared_browser_instance)

    # Navigator Agent 실행 (Blueprint 생성 지시)
    response_1 = await navigator_agent.ainvoke(
        {"messages": [HumanMessage(
            "https://news.naver.com에서 정치 섹션의 "
            "최신 기사 제목 5개와 각 기사의 상세 URL을 수집하는 Blueprint를 치밀하게 작성해 주세요. "
            "탐색은 이 URL에서 시작하되, 텍스트(제목)와 링크(URL)의 셀렉터 속성이 명확히 분리된 "
            "완벽한 Blueprint를 설계하는 것이 목표입니다."
        )]},
        config=nav_config_1,
        context=nav_context_1,
    )

    # 반환 구조 추출 및 저장
    collection_1: NavigatorBlueprintCollection = response_1["structured_response"]

    print(f"\n✅ 결과: {collection_1.total_jobs}개 Blueprint 생성")
    for i, bp in enumerate(collection_1.blueprints):
        print(f"\n  [Blueprint {i+1}]")
        print(f"    entry_urls : {bp.entry_urls}")
        print(f"    계층 수    : {bp.total_layers}")
        print(f"    렌더링 방식: {bp.rendering_type}")
        print(f"    안티봇 주의: {bp.anti_bot_notes}")
        for j, layer in enumerate(bp.layers):
            print(f"    Layer {j+1} ({layer.layer_name}) | selectors={layer.selectors}")

    print("\n📁 파일 저장:")
    paths_1 = save_blueprints(collection_1, prefix="test1_refined")

    print("\n🧹 공유 브라우저 인스턴스를 종료합니다...")
    await shared_browser_instance.stop()

if __name__ == "__main__":
    asyncio.run(main())


# Coder 에이전트 재설계

In [ ]:
from notebooks.coder import create_senior_coder, SeniorCoderContext, ARTIFACT_DIR
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# ────────────────────────────────────────────────────────
# 1. Coder 프롬프트 재정의 (웹 크롤링 특화)
# ────────────────────────────────────────────────────────
# Coder 에이전트의 내부 속성(시스템 프롬프트)을 오버라이딩하여 스크래핑 전문가로 만듭니다.
SCRAPING_CODER_PROMPT = """
당신은 최고 수준의 시니어 파이썬 웹 스크래핑 엔지니어입니다.
당신의 임무는 다른 에이전트(Navigator)가 작성한 JSON 형태의 'Blueprint(크롤링 청사진)'를 건네받아, 
그 지시대로 완벽하게 작동하는 파이썬 크롤링 스크립트를 작성하고 실행하는 것입니다.

[웹 스크래핑 핵심 행동 지침]
1. Blueprint 완벽 해석:
   - 주어진 JSON 데이터에서 `rendering_type`, `anti_bot_notes`, `layers` 등의 정보를 완벽하게 분석하세요.
   - 렌더링 방식이 "Static SSR"이면 `requests`와 `BeautifulSoup4`를 사용해 가볍게 작성하세요.
   - 렌더링 방식이 "Dynamic CSR/JS"이거나 `anti_bot_notes`에 JS 렌더링이 언급되어 있다면, 무조건 `playwright`의 
     비동기 방식(async_playwright)을 사용하여 동적 코드를 작성하세요.

2. 오류 방어 및 안티봇 우회 (Robstness):
   - `anti_bot_notes`에 경고가 있다면, User-Agent 위조(fake_useragent), 브라우저 헤더 추가, 
     요청 간 대기(time.sleep 또는 asyncio.sleep) 코드를 반드시 포함하세요.
   - CSS 셀렉터를 사용할 때, 요소가 없을 경우에 대비해 항상 `try-except` 블럭 또는 `if element is None` 
     예외 처리를 견고하게 넣어야 합니다.

3. 코드 작성 방식:
   - `create_new_file` 도구를 사용해 스크립트 파일을 생성합니다. 파일명은 `naver_news_crawler.py` 처럼 
     직관적으로 지으세요.
   - 작성된 스크립트의 맨 아래에는 `if __name__ == "__main__":` 블록을 만들고 직접 함수를 실행하는 코드를 넣으세요.
   - 파싱한 정보(예: 기사 제목 5개 등)는 화면에 `print`로 보기 좋게 출력되도록 하세요. 
     (그래야 run_python_script 툴로 당신이 결과를 볼 수 있습니다!)

4. 검증 및 디버깅:
   - 스크립트를 생성한 직후, 반드시 `run_python_script` 툴을 사용해 자신이 만든 파이썬 코드를 실행하세요.
   - 만약 에러[Error Output]가 떨어지거나 아무것도 수집되지 않는다면, 
     `read_code_file` -> `edit_code_file` 콤보를 사용해 에러가 난 라인만 수술하듯 고치고 다시 실행하세요.

당신의 최종 목표는 "에러 없이 완벽하게 데이터를 콘솔에 출력해내는 파이썬 파일 생성" 입니다.
"""

async def run_crawling_pipeline():
    print("=" * 60)
    print("🤖 Senior Coder Agent 기동 (웹 스크래핑 모드)")
    print("=" * 60)
    
    # 에이전트 초기화 후 프롬프트 교체
    coder_agent = create_senior_coder()
    coder_config = {"configurable": {"thread_id": "scraping_coder_session"}}
    coder_context = SeniorCoderContext()

    # ────────────────────────────────────────────────────────
    # 2. 이전에 Navigator가 짠 Blueprint 읽기
    # ────────────────────────────────────────────────────────
    blueprint_path = os.path.join(ARTIFACT_DIR, "blueprint_test1_1.json")
    if not os.path.exists(blueprint_path):
        print(f"❌ Blueprint 파일이 없습니다. 경로를 확인하세요: {blueprint_path}")
        return

    with open(blueprint_path, "r", encoding="utf-8") as f:
        blueprint_content = f.read()

    print("📄 Navigator의 Blueprint를 획득했습니다. Coder에게 전달합니다...\n")

    # ────────────────────────────────────────────────────────
    # 3. Coder에게 명령 하달
    # ────────────────────────────────────────────────────────
    mission_query = f"""
    다음은 Navigator 에이전트가 탐색하여 넘겨준 완벽한 크롤링 설계도(Blueprint) JSON 입니다.

    [크롤링 Blueprint JSON]
    {blueprint_content}

    [당신의 미션]
    1. 위 JSON의 'rendering_type'과 'anti_bot_notes'를 살펴보고 적절한 파이썬 라이브러리(BeautifulSoup 또는 Playwright)를 선택하세요.
    2. `create_new_file` 도구를 사용해 크롤링 스크립트(예: `naver_crawling.py`)를 작성하세요.
    (작성 시, `layers[0]` 구역의 'entry_urls'와 'selectors' 필드를 있는 그대로 파이썬 코드에 넣으세요.)
    3. `run_python_script` 도구로 작성이 끝난 스크립트를 즉시 실행하고 터미널 출력을 검증하세요.
    4. 만약 아무 기사도 안 나오거나 에러가 나면, 원인을 분석하고 디버깅하여 기사 제목이 나올 때까지 코드를 수정(edit)하고 다시 실행하세요.
    5. 미션이 완료되면 최종 결과물과 데이터를 보고해 주세요.

    ※ 참고: 시스템 프롬프트의 [웹 스크래핑 핵심 행동 지침]을 우선적으로 엄격하게 준수하세요.
    """
    # 시스템 프롬프트를 첫 메시지로 강제 주입
    messages = [
        SystemMessage(content=SCRAPING_CODER_PROMPT),
        HumanMessage(content=mission_query)
    ]

    print("⏳ Coder 가동 중 (코드 작성 및 실행)... 수십 초가 소요될 수 있습니다.")
    result = await coder_agent.ainvoke(
        {"messages": messages},
        config=coder_config,
        context=coder_context
    )
    
    # 마지막 응답 추출
    parser = StrOutputParser()
    final_response = parser.invoke(result['messages'][-1])

    print("\n" + "=" * 60)
    print("✅ Coder 작업 완료 보고서:")
    print("=" * 60)
    print(final_response)
    return final_response

if __name__ == "__main__":
    final_response = asyncio.run(run_crawling_pipeline())

### 성공!

```
HumanMessage (URL + 목표)
    ↓
Navigator 에이전트
    ├─ get_page_structure() → confidence=low (JS 렌더링 실패)
    └─ browse_web() → a.sa_text_title 확인
    ↓
NavigatorBlueprintCollection
    { selectors: {"기사 제목": "a.sa_text_title", ...} }
    ↓
Coder 에이전트
    ├─ Playwright 코드 작성
    └─ execute_python_code() 실행
    ↓
scraping_result.json ✅
```

```
[
    {
        "기사 제목": "與 '尹정권 조작기소 국조 추진위' 설치하자 '李 공취모' 탈퇴 행렬",
        "기사 URL": "https://n.news.naver.com/mnews/article/656/0000168370"
    },
    {
        "기사 제목": "강기정 광주시장 \"잠시만 광주공항에서 국제선을 운영하겠다\"",
        "기사 URL": "https://n.news.naver.com/mnews/article/421/0008793751"
    },
    {
        "기사 제목": "與, 법왜곡죄법 수정키로…\"명확성 추가해 위헌 소지 최소화\"",
        "기사 URL": "https://n.news.naver.com/mnews/article/001/0015924416"
    },
    {
        "기사 제목": "'자사주 의무소각' 3차 상법 개정안 국회 본회의 통과(상보)",
        "기사 URL": "https://n.news.naver.com/mnews/article/018/0006224624"
    },
    {
        "기사 제목": "TK 통합 불발에 국힘 ‘텃밭’ 영남권 의원들도 뿔났다",
        "기사 URL": "https://n.news.naver.com/mnews/article/005/0001833933"
    }
]


```

# 🏆 실습: LangGraph로 파이프라인 자동화하기

이번 실습에서 우리는 Navigator와 Coder를 **별도로 실행**하고 중간에 직접 Blueprint를 넘겨주었습니다.

```python
# 지금까지의 방식: 수동 연결
nav_response = await navigator_agent.ainvoke(...)   # 1단계 직접 실행
blueprint = nav_response["structured_response"]      # 중간 결과 수동 추출
coder_response = await coder_agent.ainvoke(...)      # 2단계 직접 실행
```


이 방식은 간단하지만, 한계가 있습니다.

- 사람이 중간에 개입해야 다음 단계가 실행됩니다.
- 새로운 에이전트가 추가되면 연결 코드가 계속 늘어납니다.
- 조건에 따른 분기(예: 실패 시 재시도)를 구현하기가 복잡합니다.


## 목표

LangGraph StateGraph를 사용하여 Navigator → Coder 파이프라인을 자동화하세요.
사용자가 수집 목표만 입력하면, 아래 흐름이 자동으로 실행되어야 합니다:

```
[START]
   ↓
[Navigator 노드]  - 목표 URL 탐색 → Blueprint 생성
   ↓
[Coder 노드]      - Blueprint 해석 → 코드 작성 → 실행
   ↓
[END]
```